In [34]:
import pandas as pd

merged_merchant_with_categories = pd.read_parquet('../data/curated/merged_merchant_info_with_categories.parquet')

In [35]:
merged_merchant_with_categories.head()

,merchant_abn,category_label,take_rate_value,fraud_probability,average_monthly_revenue,average_monthly_transactions
0,10023283211,Furniture and Home Furnishing,0.18,0.0,216.736529,155.285714
21,10142254217,"Media and Technology, Services",4.22,0.0,37.716001,144.571429
42,10165489824,Retail Goods,4.40,0.0,11236.094771,0.454545
53,10187291046,"Retail Goods, Services",3.29,0.0,119.526004,16.800000
73,10192359162,Retail Goods,6.33,0.0,462.329450,19.250000


In [36]:
consumer_tbl = pd.read_csv('../data/raw/tbl_consumer.csv', delimiter='|')
userdf = pd.read_parquet('../data/raw/consumer_user_details.parquet')
merged = consumer_tbl.merge(userdf, on='consumer_id', how='left')

In [37]:
test_df = pd.read_parquet('../test/')

In [38]:
merged_with_test = merged.merge(test_df, on='user_id', how='left')
merged_with_test_dropped = merged_with_test.dropna()

In [39]:
# Dropped na since no one has bought from these merchants
merged_with_abn = merged_with_test_dropped.merge(merged_merchant_with_categories, on='merchant_abn', how='left')
merged_with_abn_dropped = merged_with_abn.dropna()

In [40]:
columns_to_drop = ['name', 'address', 'state', 'gender', 'dollar_value', 'order_id', 'order_datetime', 'category_label']

cleaned_merged_with_abn = merged_with_abn_dropped.drop(columns=columns_to_drop)

In [41]:
census_data = pd.read_csv('../data/curated/2021Census_G02_AUST_POA_curated.csv')
census_data = census_data.rename(columns={'POA_CODE_2021': 'postcode'})
census_data.head()
merged_with_census = cleaned_merged_with_abn.merge(census_data, on='postcode', how='left')

print(merged_with_census.head())
merged_with_census.isnull().sum()

   postcode  consumer_id  user_id  merchant_abn  take_rate_value  \
0      6935      1195503        1  2.800049e+10             4.24   
1      6935      1195503        1  8.369064e+10             3.15   
2      6935      1195503        1  9.668077e+10             5.91   
3      6935      1195503        1  4.989171e+10             5.80   
4      6935      1195503        1  6.240384e+10             6.46   

   fraud_probability  average_monthly_revenue  average_monthly_transactions  \
0           0.000000               245.258150                    180.523810   
1           0.000000                88.512653                   1707.238095   
2          29.555245               316.660874                   1482.571429   
3           0.000000                28.889602                  11786.952381   
4           0.000000                34.467794                    184.571429   

   Median_age_persons  Median_mortgage_repay_monthly  Median_rent_weekly  \
0                 NaN                   

postcode                               0
consumer_id                            0
user_id                                0
merchant_abn                           0
take_rate_value                        0
fraud_probability                      0
average_monthly_revenue                0
average_monthly_transactions           0
Median_age_persons               2244225
Median_mortgage_repay_monthly    2244225
Median_rent_weekly               2244225
Median_tot_hhd_inc_weekly        2244225
dtype: int64

In [42]:
merged_with_census_dropped = merged_with_census.dropna()
merged_with_census_dropped.head()

,postcode,consumer_id,user_id,merchant_abn,take_rate_value,fraud_probability,average_monthly_revenue,average_monthly_transactions,Median_age_persons,Median_mortgage_repay_monthly,Median_rent_weekly,Median_tot_hhd_inc_weekly
553,2782,179208,2,8.077982e+10,4.71,0.0,34.567393,1735.142857,50.0,2080.0,430.0,1528.0
554,2782,179208,2,2.864764e+10,3.32,0.0,263.829390,23.476190,50.0,2080.0,430.0,1528.0
555,2782,179208,2,3.581349e+10,5.66,0.0,45.885892,182.761905,50.0,2080.0,430.0,1528.0
556,2782,179208,2,3.809009e+10,3.24,0.0,172.158074,2467.428571,50.0,2080.0,430.0,1528.0
557,2782,179208,2,3.897149e+10,3.61,0.0,166.642856,269.761905,50.0,2080.0,430.0,1528.0


In [43]:
aggregated_df = merged_with_census_dropped.groupby('merchant_abn').mean()
aggregated_df = aggregated_df.reset_index()
aggregated_df = aggregated_df.drop(columns=['consumer_id', 'postcode', 'user_id'])

aggregated_df.head()

,merchant_abn,take_rate_value,fraud_probability,average_monthly_revenue,average_monthly_transactions,Median_age_persons,Median_mortgage_repay_monthly,Median_rent_weekly,Median_tot_hhd_inc_weekly
0,1.002328e+10,0.18,0.0,216.736529,155.285714,43.367701,1544.717295,297.540281,1598.090540
1,1.014225e+10,4.22,0.0,37.716001,144.571429,43.285770,1566.667060,299.012972,1609.036557
2,1.016549e+10,4.40,0.0,11236.094771,0.454545,39.500000,1735.250000,332.500000,1855.250000
3,1.018729e+10,3.29,0.0,119.526004,16.800000,43.839041,1563.356164,294.119863,1595.914384
4,1.019236e+10,6.33,0.0,462.329450,19.250000,43.287926,1595.092879,309.244582,1657.421053


In [44]:
merchant_categories = pd.read_parquet('../data/curated/merchant_categories.parquet')
merchant_categories = merchant_categories.drop(columns=['category', 'revenue_rate', 'fraud_probability', 'take_rate_value'])
category_df = aggregated_df.merge(merchant_categories, on='merchant_abn', how='left')



In [45]:
merchant_fraud = pd.read_csv('../data/raw/merchant_fraud_probability.csv')
final_df = category_df.merge(merchant_fraud, on='merchant_abn', how='left')
final_df['fraud_probability_y'] = final_df['fraud_probability_y'].apply(lambda x: 0 if pd.isna(x) else 1)
final_df = final_df.drop(columns = ['order_datetime'])
final_df = final_df.rename(columns={'fraud_probability_x': 'consumer_fraud', 'fraud_probability_y': 'merchant_fraud'})
final_df2 = final_df


In [46]:
final_df.head()

,merchant_abn,take_rate_value,consumer_fraud,average_monthly_revenue,average_monthly_transactions,Median_age_persons,Median_mortgage_repay_monthly,Median_rent_weekly,Median_tot_hhd_inc_weekly,name,category_label,merchant_fraud
0,1.002328e+10,0.18,0.0,216.736529,155.285714,43.367701,1544.717295,297.540281,1598.090540,Felis Limited,Furniture and Home Furnishing,0
1,1.014225e+10,4.22,0.0,37.716001,144.571429,43.285770,1566.667060,299.012972,1609.036557,Arcu Ac Orci Corporation,Media and Technology,0
2,1.016549e+10,4.40,0.0,11236.094771,0.454545,39.500000,1735.250000,332.500000,1855.250000,Nunc Sed Company,Retail Goods,0
3,1.018729e+10,3.29,0.0,119.526004,16.800000,43.839041,1563.356164,294.119863,1595.914384,Ultricies Dignissim Lacus Foundation,Retail Goods,0
4,1.019236e+10,6.33,0.0,462.329450,19.250000,43.287926,1595.092879,309.244582,1657.421053,Enim Condimentum PC,Retail Goods,0


In [47]:
final_df['Median_age_persons'] = (25 - final_df['Median_age_persons']).abs()
final_df.head()

,merchant_abn,take_rate_value,consumer_fraud,average_monthly_revenue,average_monthly_transactions,Median_age_persons,Median_mortgage_repay_monthly,Median_rent_weekly,Median_tot_hhd_inc_weekly,name,category_label,merchant_fraud
0,1.002328e+10,0.18,0.0,216.736529,155.285714,18.367701,1544.717295,297.540281,1598.090540,Felis Limited,Furniture and Home Furnishing,0
1,1.014225e+10,4.22,0.0,37.716001,144.571429,18.285770,1566.667060,299.012972,1609.036557,Arcu Ac Orci Corporation,Media and Technology,0
2,1.016549e+10,4.40,0.0,11236.094771,0.454545,14.500000,1735.250000,332.500000,1855.250000,Nunc Sed Company,Retail Goods,0
3,1.018729e+10,3.29,0.0,119.526004,16.800000,18.839041,1563.356164,294.119863,1595.914384,Ultricies Dignissim Lacus Foundation,Retail Goods,0
4,1.019236e+10,6.33,0.0,462.329450,19.250000,18.287926,1595.092879,309.244582,1657.421053,Enim Condimentum PC,Retail Goods,0


In [48]:
cwgr = pd.read_csv('../data/curated/cwgr.csv')
final_df = final_df.merge(cwgr, on='merchant_abn', how='left')
final_df.head()

,merchant_abn,take_rate_value,consumer_fraud,average_monthly_revenue,average_monthly_transactions,Median_age_persons,Median_mortgage_repay_monthly,Median_rent_weekly,Median_tot_hhd_inc_weekly,name,category_label,merchant_fraud,cwgr
0,1.002328e+10,0.18,0.0,216.736529,155.285714,18.367701,1544.717295,297.540281,1598.090540,Felis Limited,Furniture and Home Furnishing,0,0.026743
1,1.014225e+10,4.22,0.0,37.716001,144.571429,18.285770,1566.667060,299.012972,1609.036557,Arcu Ac Orci Corporation,Media and Technology,0,0.051124
2,1.016549e+10,4.40,0.0,11236.094771,0.454545,14.500000,1735.250000,332.500000,1855.250000,Nunc Sed Company,Retail Goods,0,0.000000
3,1.018729e+10,3.29,0.0,119.526004,16.800000,18.839041,1563.356164,294.119863,1595.914384,Ultricies Dignissim Lacus Foundation,Retail Goods,0,0.000167
4,1.019236e+10,6.33,0.0,462.329450,19.250000,18.287926,1595.092879,309.244582,1657.421053,Enim Condimentum PC,Retail Goods,0,0.007906


In [49]:
final_df['bnpl_revenue'] = final_df['average_monthly_revenue'] * final_df['take_rate_value']
final_df.drop(columns=['average_monthly_revenue', 'take_rate_value'], inplace=True)


In [50]:
final_df.to_csv('../data/curated/finaldf.csv', index = False)

In [51]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

# Assuming final_df is your DataFrame

# Columns to exclude from standardization
exclude_columns = ['merchant_abn', 'consumer_fraud', 'merchant_fraud', 'category_label', 'name']

# Columns to standardize
columns_to_standardize = [col for col in final_df.columns if col not in exclude_columns]

# Initialize the StandardScaler
scaler = StandardScaler()

# Apply the scaler to the columns to be standardized
final_df[columns_to_standardize] = scaler.fit_transform(final_df[columns_to_standardize])

# Display the first few rows of the updated DataFrame to verify
final_df.head()

,merchant_abn,consumer_fraud,average_monthly_transactions,Median_age_persons,Median_mortgage_repay_monthly,Median_rent_weekly,Median_tot_hhd_inc_weekly,name,category_label,merchant_fraud,cwgr,bnpl_revenue
0,1.002328e+10,0.0,-0.011416,-0.069014,-0.083224,-0.097389,-0.064340,Felis Limited,Furniture and Home Furnishing,0,0.929887,-0.360239
1,1.014225e+10,0.0,-0.027317,-0.139663,0.045873,-0.024827,0.068302,Arcu Ac Orci Corporation,Media and Technology,0,2.354366,-0.351808
2,1.016549e+10,0.0,-0.241195,-3.404137,1.037388,1.625141,3.051869,Nunc Sed Company,Retail Goods,0,-0.632586,3.105992
3,1.018729e+10,0.0,-0.216937,0.337422,0.026400,-0.265919,-0.090710,Ultricies Dignissim Lacus Foundation,Retail Goods,0,-0.622813,-0.335384
4,1.019236e+10,0.0,-0.213301,-0.137805,0.213058,0.479304,0.654616,Enim Condimentum PC,Retail Goods,0,-0.170681,-0.157630


In [52]:
import pandas as pd

# Assuming final_df is your DataFrame

# Define the weights for each column
weights = {
    'merchant_fraud': -6,
    'cwgr': 2.5,
    'C': 2.5,
    'consumer_fraud': -2.5,
    'average_monthly_transactions': 2.3,
    'Median_tot_hhd_inc_weekly': 0.2,
    'Median_age_persons': -0.15,
    'Median_mortgage_repay_monthly': -0.1,
    'Median_rent_weekly': -0.1
}

# Calculate the score for each row
final_df['score'] = (
    final_df['merchant_fraud'] * weights['merchant_fraud'] +
    (final_df['bnpl_revenue'] - final_df['bnpl_revenue'].mean()) * weights['bnpl_revenue'] +
    final_df['consumer_fraud'] * weights['consumer_fraud'] +
    (final_df['average_monthly_transactions'] - final_df['average_monthly_transactions'].mean()) * weights['average_monthly_transactions'] +
    (final_df['Median_tot_hhd_inc_weekly'] - final_df['Median_tot_hhd_inc_weekly'].mean()) * weights['Median_tot_hhd_inc_weekly'] +
    (final_df['Median_age_persons'] - final_df['Median_age_persons'].mean()) * weights['Median_age_persons'] +
    (final_df['Median_mortgage_repay_monthly'] - final_df['Median_mortgage_repay_monthly'].mean()) * weights['Median_mortgage_repay_monthly'] +
    (final_df['Median_rent_weekly'] - final_df['Median_rent_weekly'].mean()) * weights['Median_rent_weekly'] +
    (final_df['cwgr'] - final_df['cwgr'].mean()) * weights['cwgr']
)

# Display the first few rows of the updated DataFrame to verify
final_df.head()

,merchant_abn,consumer_fraud,average_monthly_transactions,Median_age_persons,Median_mortgage_repay_monthly,Median_rent_weekly,Median_tot_hhd_inc_weekly,name,category_label,merchant_fraud,cwgr,bnpl_revenue,score
0,1.002328e+10,0.0,-0.011416,-0.069014,-0.083224,-0.097389,-0.064340,Felis Limited,Furniture and Home Furnishing,0,0.929887,-0.360239,1.413410
1,1.014225e+10,0.0,-0.027317,-0.139663,0.045873,-0.024827,0.068302,Arcu Ac Orci Corporation,Media and Technology,0,2.354366,-0.351808,4.976071
2,1.016549e+10,0.0,-0.241195,-3.404137,1.037388,1.625141,3.051869,Nunc Sed Company,Retail Goods,0,-0.632586,3.105992,6.483511
3,1.018729e+10,0.0,-0.216937,0.337422,0.026400,-0.265919,-0.090710,Ultricies Dignissim Lacus Foundation,Retail Goods,0,-0.622813,-0.335384,-2.939251
4,1.019236e+10,0.0,-0.213301,-0.137805,0.213058,0.479304,0.654616,Enim Condimentum PC,Retail Goods,0,-0.170681,-0.157630,-1.229010


In [53]:
# Columns to exclude from the calculations
exclude_columns = ['merchant_abn', 'consumer_fraud', 'merchant_fraud', 'category_label', 'name']

# Columns to include in the calculations
columns_to_include = [col for col in final_df.columns if col not in exclude_columns]

# Calculate the average, min, and max for each included column
averages = final_df[columns_to_include].mean()
minimums = final_df[columns_to_include].min()
maximums = final_df[columns_to_include].max()

# Display the results
print("Averages:")
print(averages)
print("\nMinimums:")
print(minimums)
print("\nMaximums:")
print(maximums)

Averages:
average_monthly_transactions     1.439228e-17
Median_age_persons              -9.647192e-16
Median_mortgage_repay_monthly    3.680936e-16
Median_rent_weekly               1.133938e-15
Median_tot_hhd_inc_weekly       -3.014529e-15
cwgr                            -1.308390e-16
bnpl_revenue                     6.367496e-17
score                           -2.671265e+00
dtype: float64

Minimums:
average_monthly_transactions      -0.241684
Median_age_persons               -14.182899
Median_mortgage_repay_monthly     -9.168425
Median_rent_weekly               -14.757751
Median_tot_hhd_inc_weekly         -9.656680
cwgr                              -9.777332
bnpl_revenue                      -0.362607
score                           -231.366593
dtype: float64

Maximums:
average_monthly_transactions     20.217869
Median_age_persons               15.997635
Median_mortgage_repay_monthly    49.640347
Median_rent_weekly               11.109973
Median_tot_hhd_inc_weekly        18.426334
cwg

In [54]:
import pandas as pd

# Assuming final_df is your DataFrame and the 'score' column has already been calculated

# Sort the DataFrame by the 'score' column in descending order
sorted_df = final_df.sort_values(by='score', ascending=False)

# Select the top 100 merchants
top_100_merchants = sorted_df.head(100)
top_100_merchants.to_csv('../data/curated/top_100_merchants.csv', index=False)

# Display the top 100 merchants
top_100_merchants.head(20)


,merchant_abn,consumer_fraud,average_monthly_transactions,Median_age_persons,Median_mortgage_repay_monthly,Median_rent_weekly,Median_tot_hhd_inc_weekly,name,category_label,merchant_fraud,cwgr,bnpl_revenue,score
692,2.485245e+10,0.0,20.217869,-0.042681,-0.015155,0.005401,-0.000977,Erat Vitae LLP,"Retail Goods, Furniture and Home Furnishing",0,0.745025,-0.356777,47.478903
1372,3.915015e+10,0.0,-0.240385,-6.422190,-0.887322,-2.439787,2.043061,Aliquam Eu Institute,Retail Goods,0,-0.632586,18.318869,45.367474
3478,8.657848e+10,0.0,19.027866,-0.017165,0.012455,0.053073,0.038967,Leo In Consulting,Retail Goods,0,0.739331,-0.347169,44.748312
2467,6.420342e+10,0.0,18.187323,-0.024466,0.018950,0.060854,0.046309,Pede Nonummy Corp.,Furniture and Home Furnishing,0,0.765734,-0.357167,42.857212
1854,4.989171e+10,0.0,17.250669,-0.031752,-0.000536,0.027777,0.015705,Non Vestibulum Industries,Furniture and Home Furnishing,0,0.825484,-0.351219,40.867380
1719,4.680414e+10,0.0,16.325464,-0.015284,0.010812,0.023865,0.019440,Suspendisse Dui Corporation,Services,0,0.719695,-0.356784,38.458556
1662,4.562922e+10,0.0,15.888797,-0.044448,-0.013362,0.021641,0.022286,Lacus Consulting,Retail Goods,0,0.875866,-0.345057,37.881554
3625,8.972601e+10,0.0,15.020128,-0.050592,-0.000490,0.036045,0.032474,Est Nunc Consulting,Furniture and Home Furnishing,0,0.757192,-0.345547,35.585933
1558,4.318652e+10,0.0,13.956552,-0.030164,-0.002203,0.027633,0.020718,Lorem Ipsum Sodales Industries,"Retail Goods, Furniture and Home Furnishing",0,0.840269,-0.348921,33.334564
3206,8.032405e+10,0.0,13.668715,-0.039750,0.004692,0.044481,0.015899,Ipsum Dolor Sit Corporation,Retail Goods,0,0.801552,-0.359187,32.548182


In [55]:
# Assuming top_100_merchants is your DataFrame containing the top 100 merchants

# Count the number of merchants with merchant_fraud equal to 1
num_merchants_with_fraud = (top_100_merchants['average_monthly_transactions'] > -0.2).sum()

# Display the result
print(f"Number of merchants with merchant_fraud = 1: {num_merchants_with_fraud}")

Number of merchants with merchant_fraud = 1: 59


In [56]:
import pandas as pd

# Assuming final_df is your DataFrame and the 'score' column has already been calculated

# Define the category labels
categories = ['Retail Goods', 'Media and Technology', 'Furniture and Home Furnishing', 'Services', 'Automotives']

# Initialize variables to store the top 10 merchants for each category
top_10_retail_goods = None
top_10_media_technology = None
top_10_furniture_home_furnishing = None
top_10_services = None
top_10_automotives = None

# Loop through each category and get the top 10 merchants
for category in categories:
    # Filter and sort the DataFrame by the current category and score
    top_10_merchants = final_df[final_df['category_label'].apply(lambda x: category in x)].sort_values(by='score', ascending=False).head(10)
    

    # Store the result in the corresponding variable
    if category == 'Retail Goods':
        top_10_retail_goods = top_10_merchants
    elif category == 'Media and Technology':
        top_10_media_technology = top_10_merchants
    elif category == 'Furniture and Home Furnishing':
        top_10_furniture_home_furnishing = top_10_merchants
    elif category == 'Services':
        top_10_services = top_10_merchants
    elif category == 'Automotives':
        top_10_automotives = top_10_merchants



In [57]:
top_10_retail_goods.to_csv('../data/curated/top_10_retail_goods.csv', index=False)
top_10_media_technology.to_csv('../data/curated/top_10_media_technology.csv', index=False)
top_10_furniture_home_furnishing.to_csv('../data/curated/top_10_furniture_home_furnishing.csv', index=False)
top_10_services.to_csv('../data/curated/top_10_services.csv', index=False)
top_10_automotives.to_csv('../data/curated/top_10_automotives.csv', index=False)


In [58]:
# Display the top 10 merchants for each category
print("Top 10 merchants in 'Retail Goods':")
print(top_10_retail_goods)
print("\n")

print("Top 10 merchants in 'Media and Technology':")
print(top_10_media_technology)
print("\n")

print("Top 10 merchants in 'Furniture and Home Furnishing':")
print(top_10_furniture_home_furnishing)
print("\n")

print("Top 10 merchants in 'Services':")
print(top_10_services)
print("\n")

print("Top 10 merchants in 'Automotives':")
print(top_10_automotives)
print("\n")

Top 10 merchants in 'Retail Goods':
      merchant_abn  consumer_fraud  average_monthly_transactions  \
692   2.485245e+10             0.0                     20.217869   
1372  3.915015e+10             0.0                     -0.240385   
3478  8.657848e+10             0.0                     19.027866   
1662  4.562922e+10             0.0                     15.888797   
1558  4.318652e+10             0.0                     13.956552   
3206  8.032405e+10             0.0                     13.668715   
16    1.040454e+10             0.0                     -0.240385   
895   2.906845e+10             0.0                     -0.240385   
2058  5.486013e+10             0.0                     -0.239495   
1924  5.142505e+10             0.0                     -0.241539   

      Median_age_persons  Median_mortgage_repay_monthly  Median_rent_weekly  \
692            -0.042681                      -0.015155            0.005401   
1372           -6.422190                      -0.887322  

In [59]:
import pandas as pd

# Assuming top_100_merchants and final_df2 are your DataFrames

# Select only the 'merchant_abn' and 'score' columns from top_100_merchants
top_100_merchants_reduced = top_100_merchants[['merchant_abn', 'score']]

# Merge with final_df2 on 'merchant_abn', keeping only the merchants from top_100_merchants
top_100_merged = pd.merge(top_100_merchants_reduced, final_df2, on='merchant_abn', how='left')

# Display the first few rows of the merged DataFrame to verify
top_100_merged.head(10)

,merchant_abn,score,take_rate_value,consumer_fraud,average_monthly_revenue,average_monthly_transactions,Median_age_persons,Median_mortgage_repay_monthly,Median_rent_weekly,Median_tot_hhd_inc_weekly,name,category_label,merchant_fraud
0,2.485245e+10,47.478903,2.94,0.0,30.052613,13786.333333,18.398239,1556.290754,299.626473,1603.319408,Erat Vitae LLP,"Retail Goods, Furniture and Home Furnishing",0
1,3.915015e+10,45.367474,6.99,0.0,38089.939272,1.000000,11.000000,1408.000000,250.000000,1772.000000,Aliquam Eu Institute,Retail Goods,0
2,8.657848e+10,44.748312,6.43,0.0,35.036088,12984.476190,18.427831,1560.985112,300.593985,1606.615735,Leo In Consulting,Retail Goods,0
3,6.420342e+10,42.857212,2.86,0.0,28.949356,12418.095238,18.419364,1562.089539,300.751919,1607.221653,Pede Nonummy Corp.,Furniture and Home Furnishing,0
4,4.989171e+10,40.867380,5.80,0.0,28.889602,11786.952381,18.410914,1558.776322,300.080589,1604.696067,Non Vestibulum Industries,Furniture and Home Furnishing,0
5,4.680414e+10,38.458556,2.93,0.0,30.118585,11163.523810,18.430012,1560.705850,300.001206,1605.004309,Suspendisse Dui Corporation,Services,0
6,4.562922e+10,37.881554,6.98,0.0,36.588292,10869.285714,18.396191,1556.595595,299.956066,1605.239186,Lacus Consulting,Retail Goods,0
7,8.972601e+10,35.585933,6.01,0.0,41.329641,10283.952381,18.389066,1558.784247,300.248403,1606.079958,Est Nunc Consulting,Furniture and Home Furnishing,0
8,4.318652e+10,33.334564,4.47,0.0,44.812806,9567.285714,18.412755,1558.492916,300.077676,1605.109747,Lorem Ipsum Sodales Industries,"Retail Goods, Furniture and Home Furnishing",0
9,8.032405e+10,32.548182,1.47,0.0,36.733984,9373.333333,18.401638,1559.665325,300.419605,1604.712101,Ipsum Dolor Sit Corporation,Retail Goods,0


In [60]:
import pandas as pd

# Assuming final_df2 is your DataFrame and the top 10 DataFrames are already defined

# Function to merge and keep only 'score' column from top merchants DataFrame
def merge_top_merchants(top_merchants_df, final_df2):
    # Select only the 'merchant_abn' and 'score' columns from top_merchants_df
    top_merchants_reduced = top_merchants_df[['merchant_abn', 'score']]
    
    # Merge with final_df2 on 'merchant_abn', keeping only the merchants from top_merchants_df
    merged_df = pd.merge(top_merchants_reduced, final_df2, on='merchant_abn', how='left')
    
    return merged_df

# Merge for each top 10 merchants DataFrame
merged_top_10_retail_goods = merge_top_merchants(top_10_retail_goods, final_df2)
merged_top_10_media_technology = merge_top_merchants(top_10_media_technology, final_df2)
merged_top_10_furniture_home_furnishing = merge_top_merchants(top_10_furniture_home_furnishing, final_df2)
merged_top_10_services = merge_top_merchants(top_10_services, final_df2)
merged_top_10_automotives = merge_top_merchants(top_10_automotives, final_df2)

# Display the first few rows of each merged DataFrame to verify
print("Merged Top 10 Retail Goods:")
print(merged_top_10_retail_goods.head())
print("\n")

print("Merged Top 10 Media and Technology:")
print(merged_top_10_media_technology.head())
print("\n")

print("Merged Top 10 Furniture and Home Furnishing:")
print(merged_top_10_furniture_home_furnishing.head())
print("\n")

print("Merged Top 10 Services:")
print(merged_top_10_services.head())
print("\n")

print("Merged Top 10 Automotives:")
print(merged_top_10_automotives.head())
print("\n")

Merged Top 10 Retail Goods:
   merchant_abn      score  take_rate_value  consumer_fraud  \
0  2.485245e+10  47.478903             2.94             0.0   
1  3.915015e+10  45.367474             6.99             0.0   
2  8.657848e+10  44.748312             6.43             0.0   
3  4.562922e+10  37.881554             6.98             0.0   
4  4.318652e+10  33.334564             4.47             0.0   

   average_monthly_revenue  average_monthly_transactions  Median_age_persons  \
0                30.052613                  13786.333333           18.398239   
1             38089.939272                      1.000000           11.000000   
2                35.036088                  12984.476190           18.427831   
3                36.588292                  10869.285714           18.396191   
4                44.812806                   9567.285714           18.412755   

   Median_mortgage_repay_monthly  Median_rent_weekly  \
0                    1556.290754          299.626473   


In [61]:
merged_top_10_retail_goods.head()

,merchant_abn,score,take_rate_value,consumer_fraud,average_monthly_revenue,average_monthly_transactions,Median_age_persons,Median_mortgage_repay_monthly,Median_rent_weekly,Median_tot_hhd_inc_weekly,name,category_label,merchant_fraud
0,2.485245e+10,47.478903,2.94,0.0,30.052613,13786.333333,18.398239,1556.290754,299.626473,1603.319408,Erat Vitae LLP,"Retail Goods, Furniture and Home Furnishing",0
1,3.915015e+10,45.367474,6.99,0.0,38089.939272,1.000000,11.000000,1408.000000,250.000000,1772.000000,Aliquam Eu Institute,Retail Goods,0
2,8.657848e+10,44.748312,6.43,0.0,35.036088,12984.476190,18.427831,1560.985112,300.593985,1606.615735,Leo In Consulting,Retail Goods,0
3,4.562922e+10,37.881554,6.98,0.0,36.588292,10869.285714,18.396191,1556.595595,299.956066,1605.239186,Lacus Consulting,Retail Goods,0
4,4.318652e+10,33.334564,4.47,0.0,44.812806,9567.285714,18.412755,1558.492916,300.077676,1605.109747,Lorem Ipsum Sodales Industries,"Retail Goods, Furniture and Home Furnishing",0


In [62]:
merged_top_10_media_technology.head()

,merchant_abn,score,take_rate_value,consumer_fraud,average_monthly_revenue,average_monthly_transactions,Median_age_persons,Median_mortgage_repay_monthly,Median_rent_weekly,Median_tot_hhd_inc_weekly,name,category_label,merchant_fraud
0,6.821691e+10,29.735778,3.05,0.0,38.604636,8610.714286,18.428784,1561.492133,300.728871,1606.650523,Placerat Eget Venenatis Limited,"Media and Technology, Furniture and Home Furni...",0
1,7.247291e+10,17.284743,6.33,0.0,70.011718,4847.190476,18.455684,1554.652268,299.542907,1603.243406,Nullam Consulting,"Retail Goods, Media and Technology",0
2,4.950593e+10,11.897322,4.70,0.0,105.355358,3249.523810,18.435244,1560.468380,300.351211,1604.653856,Suspendisse Ac Associates,"Retail Goods, Media and Technology",0
3,4.708641e+10,9.284432,2.52,0.0,11.301957,2595.190476,18.398954,1558.420866,299.889103,1602.573202,Eros Limited,"Retail Goods, Media and Technology",0
4,6.269403e+10,8.309651,5.76,0.0,26.048787,2311.190476,18.396107,1553.242741,299.178331,1602.098281,Vel Est Tempor LLP,"Media and Technology, Furniture and Home Furni...",0


In [63]:
merged_top_10_furniture_home_furnishing.head()

,merchant_abn,score,take_rate_value,consumer_fraud,average_monthly_revenue,average_monthly_transactions,Median_age_persons,Median_mortgage_repay_monthly,Median_rent_weekly,Median_tot_hhd_inc_weekly,name,category_label,merchant_fraud
0,2.485245e+10,47.478903,2.94,0.0,30.052613,13786.333333,18.398239,1556.290754,299.626473,1603.319408,Erat Vitae LLP,"Retail Goods, Furniture and Home Furnishing",0
1,6.420342e+10,42.857212,2.86,0.0,28.949356,12418.095238,18.419364,1562.089539,300.751919,1607.221653,Pede Nonummy Corp.,Furniture and Home Furnishing,0
2,4.989171e+10,40.867380,5.80,0.0,28.889602,11786.952381,18.410914,1558.776322,300.080589,1604.696067,Non Vestibulum Industries,Furniture and Home Furnishing,0
3,8.972601e+10,35.585933,6.01,0.0,41.329641,10283.952381,18.389066,1558.784247,300.248403,1606.079958,Est Nunc Consulting,Furniture and Home Furnishing,0
4,4.318652e+10,33.334564,4.47,0.0,44.812806,9567.285714,18.412755,1558.492916,300.077676,1605.109747,Lorem Ipsum Sodales Industries,"Retail Goods, Furniture and Home Furnishing",0


In [64]:
merged_top_10_services.head()

,merchant_abn,score,take_rate_value,consumer_fraud,average_monthly_revenue,average_monthly_transactions,Median_age_persons,Median_mortgage_repay_monthly,Median_rent_weekly,Median_tot_hhd_inc_weekly,name,category_label,merchant_fraud
0,4.680414e+10,38.458556,2.93,0.0,30.118585,11163.523810,18.430012,1560.705850,300.001206,1605.004309,Suspendisse Dui Corporation,Services,0
1,6.329052e+10,30.097351,6.48,0.0,24.132288,8660.761905,18.413703,1562.768270,300.133731,1606.952119,Vehicula Pellentesque Corporation,Services,0
2,4.125180e+10,8.022544,2.91,0.0,29.275393,1923.666667,18.509095,1551.298529,299.295134,1600.657946,Ultricies Sem Limited,Services,0
3,4.588840e+10,7.986315,5.71,0.0,35.946373,42.952381,18.353333,1569.453333,301.457333,1610.293333,Nulla Vulputate LLC,Services,0
4,1.156679e+10,7.622785,2.62,0.0,30.347487,2124.476190,18.421251,1560.016252,300.996578,1607.632558,Euismod Et Institute,Services,0


In [65]:
merged_top_10_automotives.head()

,merchant_abn,score,take_rate_value,consumer_fraud,average_monthly_revenue,average_monthly_transactions,Median_age_persons,Median_mortgage_repay_monthly,Median_rent_weekly,Median_tot_hhd_inc_weekly,name,category_label,merchant_fraud
0,1.277110e+10,4.969724,6.95,0.0,3700.267607,63.809524,18.453712,1551.068744,295.221815,1605.842346,At Pretium Corp.,Automotives,0
1,8.771709e+10,4.341833,6.43,0.0,309.714029,183.523810,18.336751,1558.924511,301.846226,1601.752408,Lacinia Mattis Corp.,Automotives,0
2,2.135918e+10,4.293705,3.60,0.0,313.989404,983.380952,18.340847,1558.611636,300.995538,1605.603755,Sit Amet PC,Automotives,0
3,2.026949e+10,4.211595,2.55,0.0,2915.198810,83.857143,18.275650,1569.560192,301.290014,1598.952804,Mi Eleifend Egestas Corp.,Automotives,0
4,2.203336e+10,3.906631,3.10,0.0,378.240346,1117.523810,18.456020,1563.757728,299.873856,1602.247814,Suspendisse Non Leo PC,Automotives,0
